In [ ]:
# Data download and unzipping for stage2
%%capture
!pip install kaggle
!mv kaggle.json .kaggle/
!kaggle competitions download -c data-science-bowl-2018 -f stage2_test_final.zip

!mkdir stage2_test_final
!unzip .kaggle/competitions/data-science-bowl-2018/stage2_test_final.zip -d stage2_test_final/

In [ ]:
# Data download and unzipping for stage1
%%capture
!wget https://raw.githubusercontent.com/AakashSudhakar/2018-data-science-bowl/master/compressed_files/stage1_test.zip -c
!wget https://raw.githubusercontent.com/AakashSudhakar/2018-data-science-bowl/master/compressed_files/stage1_train.zip -c

!mkdir stage1_train stage1_test
!unzip stage1_train.zip -d stage1_train/
!unzip stage1_test.zip -d stage1_test/

In [ ]:
from google.colab import files
uploaded = files.upload() # upload kaggle.json .py files

In [ ]:
%%capture
!pip install keras --upgrade

In [ ]:
import os
import random
import sys
import warnings
import numpy as np
import pandas as pd
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from keras.models import Model, load_model
from keras import optimizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore', category=UserWarning, module='skimage')

# Setting seed for reproducability
SEED = 42
np.random.seed(SEED)
%matplotlib inline

In [ ]:
from data_utils import read_train_data, read_test_data, rle_encoding, mask_to_rle
from model import get_unet, iou_metric, iou_metric_batch, my_iou_metric

In [ ]:
# get train_data
X_train,Y_train = read_train_data()
# get test_data
X_test,test_img_sizes = read_test_data()

In [ ]:
#@title Hyperparameter { run: "auto", display-mode: "both" }
BATCH_SIZE = 32 #@param {type:"raw"}
EPOCHS = 50 #@param {type:"raw"}
LEARNING_RATE = 1e-3 #@param {type:"raw"}
OPTIMIZER = optimizers.Adam(lr=LEARNING_RATE) #@param {type:"raw"}
val_size = 0.25 #@param {type:"raw"}

### KFold Model

In [ ]:
from sklearn.model_selection import KFold
k = 4
kf = KFold(n_splits=k, shuffle=True, random_state=SEED)
historys = []
train_ious, val_ious = [], []
test_pred = []
final_pred = np.zeros(X_test.shape)

for i, (train_index, val_index) in enumerate(kf.split(X_train)):
    
    train_img, train_mask = X_train[train_index], Y_train[train_index]
    val_img, val_mask = X_train[val_index], Y_train[val_index]
    u_net = get_unet(OPTIMIZER=OPTIMIZER)
    filepath= 'best_model' + str(i) + '.h5'
    print('Training...')
    earlystopper = EarlyStopping(monitor='val_my_iou_metric', patience=10, mode='max')
    checkpointer = ModelCheckpoint(filepath, monitor='val_my_iou_metric', 
                                   verbose=1, save_best_only=True, mode='max')
    result = u_net.fit(x=train_img, y=train_mask, validation_data=(val_img, val_mask), 
                       batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1, 
                       callbacks=[earlystopper, checkpointer])
    historys.append(result)
    best_model = load_model(filepath, custom_objects={'my_iou_metric': my_iou_metric})
    print('Predicting...')
    train_pred = best_model.predict(train_img, verbose=1)
    val_pred = best_model.predict(val_img, verbose=1)
    test_pred = best_model.predict(X_test, verbose=1)
    train_iou = iou_metric_batch(train_mask, train_pred)
    val_iou = iou_metric_batch(val_mask, val_pred)
    final_pred += test_pred
    # test_preds.append(test_pred)
    train_ious.append(train_iou)
    val_ious.append(val_iou)
    print('Best train IOU = ', train_iou)
    print('Best val IOU = ', val_iou) 

### Kold Submission

In [ ]:
final_pred = final_pred / 4.0

In [ ]:
# Create list of upsampled test masks
test_mask_upsampled = []
for i in range(len(final_pred)):
    test_mask_upsampled.append(resize(np.squeeze(final_pred[i]),
                                       (test_img_sizes[i][0],test_img_sizes[i][1]), 
                                       mode='constant', preserve_range=True))

In [ ]:
test_ids,rles = mask_to_rle(test_mask_upsampled)

In [ ]:
# Create submission DataFrame
sub = pd.DataFrame()
sub['ImageId'] = test_ids
sub['EncodedPixels'] = pd.Series(rles).apply(lambda x: ' '.join(str(y) for y in x))

sub.to_csv('sub-dsbowl2018.csv', index=False)

### Train/Val split Submission

In [ ]:
# Code to download files from Google colab

from google.colab import files
files.download('sub-dsbowl2018.csv')
# files.download('best_model.h5')